## Genetic Algorithm and Cultural Algorithm

In [47]:
import numpy as np
import pandas as pd 
from sklearn.metrics import accuracy_score
import warnings
import random

warnings.filterwarnings("ignore")

In [48]:
data = pd.read_csv(r"C:\Users\Lenovo\Desktop\Machine Learning Lab\Bank_Personal_Loan_Modelling.csv")
data.drop(['ID','ZIP Code'],axis=1,inplace=True)
X = data.drop(['Personal Loan'],axis=1)
Y = data['Personal Loan']
global x 
x = X.to_numpy()
global y 
y = Y.to_numpy()

Structure Of Neural Network
11->16->1

In [49]:
#Activation Function
def sigmoid (a):
    return 1/(1 + np.exp(-a))

def step(x):
    if (x >= 0.5):
        return 1  
    else:
        return 0

#Forward Propagation
def forward(weights):
    w1 = weights[:176].reshape(11,16)
    w2 = weights[176:192].reshape(16,1)
    b1 = weights[192:208].reshape(1,16)
    b2 = weights[208].reshape(1,1)
    
    hidden1 = sigmoid(np.dot(x,w1)+b1)
    output = sigmoid(np.dot(hidden1,w2)+b2)
    
    pred = np.vectorize(step)(output)
    return pred
   
#fitness/accuracy function
def fitness(weights):
    pred = forward(weights)
    acc = accuracy_score(y,pred)
    return acc*100

In [70]:
#creating population
def populator(n):
    population = []
    for i in range(n):
        parameters = np.random.randn(209)
        population.append(parameters)
    return population

#Crossover 
def crossover(parent1,parent2):
    children = []
    crossover_point = np.random.randint(low=0, high=209)
    
    children.append(np.concatenate((parent1[:crossover_point],parent2[crossover_point:]),axis=0))
    children.append(np.concatenate((parent2[:crossover_point],parent1[crossover_point:]),axis=0))
    
    return children

def mutator(gene):
    mutated = gene.copy()
    mutating_val = 0.6
    
    for i in range(20):
        l = random.randint(0,1)
        mutation_point = np.random.randint(low=0, high=209)
        
        if l == 0:
            mutated[mutation_point] += mutating_val
        else:
            mutated[mutation_point] -= mutating_val
    
    return mutated

#Implementing culutural algorithm
def belief_space(population):
    ranking = sorted(population, key=lambda rank: fitness(rank), reverse=True)
    elite = ranking[:4]
    return elite
    
def influence(population):
    ranking = sorted(population, key=lambda rank: fitness(rank), reverse=True)
    return crossover(ranking[0],ranking[1])
    
def new_populator(population):
    ranking = sorted(population, key=lambda rank: fitness(rank), reverse=True)
    elite = belief_space(population)
    new_pop= []
    #Adding the best 4 genes to next population
    new_pop.extend(elite)
    
    #Adding children of the best 2 genes(influence)
    new_pop.extend(influence(population))
    
    #Adding children of 2 random genes
    parent1, parent2 = random.sample(ranking[4:],2)
    new_pop.extend(crossover(parent1, parent2))
    
    #Mutating 2 random genes
    gene1,gene2 = random.sample(ranking[4:],2)
    new_pop.append(mutator(gene1))
    new_pop.append(mutator(gene2))
    
    return new_pop

In [83]:
population = populator(10)
count = 0 

while True:
    if fitness(best) > 98 or count == 50:
        break
    else:
        population = new_populator(population)
        ranking = sorted(population, key=lambda rank: fitness(rank), reverse=True)
        best = ranking[9]
        count +=1 

print('Final accuracy is '+ str(fitness(best)) + '%')

Final accuracy is 87.78%
